## AI-Powered System for Managing Vendor Packages

Este projeto Python foi desenvolvido para auxiliar no gerenciamento e controle de pacotes de equipamentos dos FPSOs P84 e P85.


### Importa modulos

In [1]:
import pandas as pd
from pathlib import Path
import re
from datetime import datetime

#### Ignorando as mensagens de aviso ("Deprecation Warning") no python

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.worksheet._reader")

### Importando arquivos

In [3]:
# Set the correct path to the Excel file
dados_basicos = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\DadosBasicos\\dados_basicos.xlsx'
ld_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\LD_consolidada_have_a_comment.xlsx'
mapa_topsideP84_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\P84 Procurement Map Topside.xlsx'
mapa_topsideP85_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\P85 Procurement Map Topside.xlsx'
mapa_orbis_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\orbis.xlsx'
comunication_matrix_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2024-03-10 - P84-P85 Package Communication Matrix.xlsx'
integra_eng_docs = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\RE-General Query - Technical Engineering Documents.xlsx'
vda_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\VDA\\vda.xlsx'
avanco_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\Avanço Pacotes.xlsx'
notes_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\notes.xlsx'

# Read the Excel file
df_db = pd.read_excel(dados_basicos, sheet_name='DadosBasicos')
df_all_equipment = pd.read_excel(dados_basicos, sheet_name='Equipamentos')
df_LD = pd.read_excel(ld_file, sheet_name="VDRL")
df_mapa_topsideP84 = pd.read_excel(mapa_topsideP84_file, sheet_name="P84 Topside Map")
df_mapa_topsideP85 = pd.read_excel(mapa_topsideP85_file, sheet_name="P85 Topside Map")
df_mapa_orbis = pd.read_excel(mapa_orbis_file, sheet_name="Export")
df_mc = pd.read_excel(comunication_matrix_file, sheet_name="Package Matrix")
df_integra = pd.read_excel(integra_eng_docs, sheet_name="DADOS")
# cria um dataframe chamado df_integra_last somente com os documentos na última versão:
df_integra_last = df_integra.loc[df_integra.groupby('CODE')['VERSION'].idxmax()]
df_VDA = pd.read_excel(vda_file, sheet_name="Sheet1")
df_avanco = pd.read_excel(avanco_file, sheet_name="PREENCHIMENTO AVANÇO")
df_notes = pd.read_excel(notes_file, sheet_name="notes")


#### Pré-Processamento dos dados

In [4]:
# Preparação dos DataFrames para limpeza de espaços em branco

df_db = df_db.fillna('')
df_mapa_orbis.rename(columns = {"PKG DESCRIPTION":"PKG_DESCRIPTION", "Pkg ID":"Pkg_ID", "PO Issue Plan":"PO_Issue_Date_Plan", "Last deliv. Plan/Forecast":"Required_On_Site_Date_Plan"}, inplace = True)
df_mc.rename(columns={"Package Engineer_Petrobras":"Package_Engineer_Petrobras"}, inplace = True)
df_mc.rename(columns={"Package Engineer_Engineering":"Package_Engineer_Engineering"}, inplace = True)
df_mc.rename(columns={"EPC_Responsible_(Expeditor)":"Package_Expeditor"}, inplace = True)
df_mc.rename(columns={"Package Engineer_KBR":"Package_KBR"}, inplace = True)
df_avanco.rename(columns={"PKG DESCRIPTION":"PKG_DESCRIPTION", "%PLAN":"PLAN", "%REAL":"REAL", "AVANÇO":"DESVIO"}, inplace = True)
df_avanco = df_avanco.drop(columns=['Unnamed: 6'])

# limpeza de espaços em branco para todas as colunas de cada DataFrame da lista abaixo:
# Lista de DataFrames
dataframes = [df_LD, df_mapa_topsideP84, df_mapa_topsideP85, df_mapa_orbis, df_mc]

# Função para limpar espaços em branco
def limpar_espacos(df):
    return df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

def limpar_espacos__entre_palavras(df):
    return df.applymap(lambda x: re.sub(' +', ' ', str(x)) if isinstance(x, str) else x)

# Aplicar a limpeza para cada DataFrame na lista
for i, df in enumerate(dataframes):
    dataframes[i] = limpar_espacos(df)

# Atribuir os DataFrames limpos de volta às variáveis originais 
df_LD, df_mapa_topsideP84, df_mapa_topsideP85, df_mapa_orbis, df_mc = dataframes
 
print("Limpeza de espaços em branco concluída para todos os DataFrames.")

# Aplicar a limpeza para cada DataFrame na lista
for i, df in enumerate(dataframes):
    dataframes[i] = limpar_espacos__entre_palavras(df)
 
# Atribuir os DataFrames limpos de volta às variáveis originais 
df_LD, df_mapa_topsideP84, df_mapa_topsideP85, df_mapa_orbis, df_mc = dataframes

print("Limpeza de espaços em branco concluída para todos os DataFrames (entre palavras).")

print("Ajustando o dataframe df_avanco para remover '- HULL' e - 'TOPSIDE.'")
df_avanco['PKG_DESCRIPTION'] = df_avanco['PKG_DESCRIPTION'].str.replace(' - HULL', '').str.replace(' - TOPSIDE', '')


Limpeza de espaços em branco concluída para todos os DataFrames.
Limpeza de espaços em branco concluída para todos os DataFrames (entre palavras).
Ajustando o dataframe df_avanco para remover '- HULL' e - 'TOPSIDE.'


### Scope of Supply

In [5]:
def count_qty(df):
    count = 0
    for qty_str in df['QTY']:
        parts = qty_str.split()
        # Verificar se a primeira parte é um número válido
        if parts[0].isdigit():
            count += int(parts[0])

    return count

# Group df_all_equipment by PKG_DESCRIPTION
df_grouped = df_all_equipment.groupby('PKG_DESCRIPTION')

# Create empty list to store results
data_for_df_equip = []

total = 0
for pkg_name, pkg_group in df_grouped:
    # Apply count_qty function to each group
    qty_count = count_qty(pkg_group)
    
    print(f"Total de equipamentos do pacote {pkg_name}: {qty_count}")
    total += qty_count
    
    # Add data for the new DataFrame
    data_for_df_equip.append({
        'PKG_DESCRIPTION': pkg_name, 
        'QTY_EQUIPMENT': qty_count
    })

print(f"\nTotal de equipamentos: {total}")

# Create the new DataFrame df_equip
df_equip = pd.DataFrame(data_for_df_equip)

Total de equipamentos do pacote API 610 CENTRIFUGAL PUMPS: 90
Total de equipamentos do pacote DECK TROLLEY: 10
Total de equipamentos do pacote FLARE SYSTEM: 8
Total de equipamentos do pacote FRESH WATER CHLORINATION UNIT: 2
Total de equipamentos do pacote FRESH WATER MAKER FOR OIL DILUTION: 6
Total de equipamentos do pacote NITROGEN GENERATOR UNITS: 152
Total de equipamentos do pacote NON-API 610 CENTRIFUGAL PUMPS: 17
Total de equipamentos do pacote OFFSHORE CRANE: 4
Total de equipamentos do pacote PIG LAUNCHERS AND RECEIVERS: 27
Total de equipamentos do pacote PRINTED CIRCUIT HEAT EXCHANGER: 16
Total de equipamentos do pacote PROGRESSIVE CAVITY PUMPS: 8
Total de equipamentos do pacote RECIPROCATING PUMPS: 26
Total de equipamentos do pacote SEA WATER ELECTROCHLORINATION UNIT: 2
Total de equipamentos do pacote SEA WATER LIFT PUMP: 21
Total de equipamentos do pacote SHELL AND TUBE HEAT EXCHANGERS (Asvotec): 27
Total de equipamentos do pacote SHELL AND TUBE HEAT EXCHANGERS (Himile): 6
Tot

### Função para extrair sistemas e trigramas do dataframe df_db

In [6]:
import pandas as pd
import re

# Função para extrair sistemas e trigramas do dataframe df_db

def extrair_sistemas_trigramas(df):
    # Inicializar um dicionário para armazenar relações sistema-trigrama
    relacoes = {}
    
    # Iterar sobre cada linha do dataframe
    for _, row in df.iterrows():
        # Obter o trigrama para esta linha
        trigram = row['Trigram']
        
        # Limpar e dividir a string de sistemas
        systems_str = row['System']
        # Substituir espaços antes ou depois de vírgulas
        systems_str = re.sub(r'\s*,\s*', ',', systems_str)
        # Dividir a string pelos separadores de vírgula
        systems = systems_str.split(',')
        
        # Para cada sistema nesta linha, associar ao trigrama
        for system in systems:
            system = system.strip()
            if system:  # Verificar se o sistema não está vazio
                if system not in relacoes:
                    relacoes[system] = set()
                relacoes[system].add(trigram)
    
    # Converter o dicionário para o formato de lista de tuplas
    sistemas_trigramas = []
    for system, trigrams in relacoes.items():
        for trigram in trigrams:
            sistemas_trigramas.append((system, trigram))
    
    # Ordenar a lista para melhor legibilidade
    sistemas_trigramas.sort()
    
    return sistemas_trigramas


# Extrair os sistemas e trigramas
sistemas_trigramas = extrair_sistemas_trigramas(df_db)
'''
# Imprimir o resultado formatado como no exemplo
print("# Dicionário com os sistemas e trigramas")
print("sistemas_trigramas = [")
for sistema, trigrama in sistemas_trigramas:
    print(f"    ('{sistema}', '{trigrama}'),")
print("    # Adicione mais sistemas e trigramas conforme necessário")
print("]")
'''


'\n# Imprimir o resultado formatado como no exemplo\nprint("# Dicionário com os sistemas e trigramas")\nprint("sistemas_trigramas = [")\nfor sistema, trigrama in sistemas_trigramas:\n    print(f"    (\'{sistema}\', \'{trigrama}\'),")\nprint("    # Adicione mais sistemas e trigramas conforme necessário")\nprint("]")\n'

### MAPA DE SUPRIMENTOS DO TOPSIDE:
- Percorre os 2 dataframes e atualiza o df_escopo

In [7]:
lista_dfs = ["df_mapa_topsideP84", "df_mapa_topsideP85"]

print("Atualizando informações dos Mapas de Suprimentos...")
for item in lista_dfs:
    df = eval(item)  # Converte o nome da variável em um objeto DataFrame
    print(f"Processing DataFrame: {item}")
    
    for row in df.itertuples():
        pkg_description = row.PKG_DESCRIPTION
        po_number = row.PO_Number
        # usando a função .loc para atualizar os valores
        if po_number:
            df_db.loc[(df_db['PKG_DESCRIPTION'] == pkg_description), "PO_Number"] = po_number

print("Atualizando informações do Orbis...")
for row in df_mapa_orbis.itertuples():
    pkg_description = row.PKG_DESCRIPTION
    pkg_ID = row.Pkg_ID
    PO_Issue_Date_Plan = row.PO_Issue_Date_Plan
    Required_On_Site_Date_Plan = row.Required_On_Site_Date_Plan
    # if pkg_ID:
    df_db.loc[(df_db['PKG_DESCRIPTION'] == pkg_description), "PO_Issue_Date_Plan"] = PO_Issue_Date_Plan
    df_db.loc[(df_db['PKG_DESCRIPTION'] == pkg_description), "Required_On_Site_Date_Plan"] = Required_On_Site_Date_Plan

print("Atualizando informações do Communication Matrix...")
for row in df_mc.itertuples():
    pkg_description = row.PKG_DESCRIPTION
    Package_KBR = row.Package_KBR
    Package_Seatrium = row.Package_Engineer_Engineering
    Package_Expeditor = row.Package_Expeditor
    # if pkg_ID:
    df_db.loc[(df_db['PKG_DESCRIPTION'] == pkg_description), "Package_KBR"] = Package_KBR
    df_db.loc[(df_db['PKG_DESCRIPTION'] == pkg_description), "Package_Seatrium"] = Package_Seatrium
    df_db.loc[(df_db['PKG_DESCRIPTION'] == pkg_description), "Package_Expeditor"] = Package_Expeditor

print("Atualiza informações do Integra...")

for row in df_integra_last.itertuples():
    document_emitido = row.CODE
    
    # Processa MR_Number que podem ter múltiplos valores
    for index, mr_value in df_db['MR_Number'].items():
        if pd.notna(mr_value):  # Verifica se o valor não é NaN
            # Divide os valores separados por vírgula e remove espaços
            mr_list = [item.strip() for item in str(mr_value).split(',')]
            # Verifica se o documento emitido está na lista
            if document_emitido in mr_list:
                df_db.loc[index, "MR_Emitida"] = "Sim"
    
    # Processa TBE_Number que podem ter múltiplos valores
    for index, tbe_value in df_db['TBE_Number'].items():
        if pd.notna(tbe_value):  # Verifica se o valor não é NaN
            # Divide os valores separados por vírgula e remove espaços
            tbe_list = [item.strip() for item in str(tbe_value).split(',')]
            # Verifica se o documento emitido está na lista
            if document_emitido in tbe_list:
                df_db.loc[index, "TBE_Emitida"] = "Sim"

print("Atualiza informações das quantidades de equipamentos...")
# Update df_db with equipment quantities from df_equip
for index, row in df_equip.iterrows():
    pkg_description = row['PKG_DESCRIPTION']
    qty_equipment = row['QTY_EQUIPMENT']
    
    # Update the corresponding rows in df_db
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'QTY_EQUIPMENT'] = qty_equipment

print("Atualiza informações do Avanço Geral...")
# Substituir NaN por 0 nas colunas abaixo
df_avanco['PLAN'] = df_avanco['PLAN'].fillna(0)
df_avanco['REAL'] = df_avanco['REAL'].fillna(0)
df_avanco['DESVIO'] = df_avanco['DESVIO'].fillna(0)

# Filtra apenas os registros do FPSO P84
df_avanco_p84 = df_avanco[df_avanco['FPSO'] == 'P84']
# Ordena por data em ordem crescente para garantir que o último valor seja o mais recente
df_avanco_p84 = df_avanco_p84.sort_values(by='DATE')
# Obtém os últimos valores para cada PKG_DESCRIPTION
df_latest_p84 = df_avanco_p84.drop_duplicates(subset=['PKG_DESCRIPTION'], keep='last')
# Update df_db com os valores mais recentes
for index, row in df_latest_p84.iterrows():
    pkg_description = row['PKG_DESCRIPTION']
    plan = row['PLAN']
    real = row['REAL']
    desvio = row['DESVIO']
    # Update the corresponding rows in df_db
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'PLAN_P84'] = plan
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'REAL_P84'] = real
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'DESVIO_P84'] = desvio

# Filtra apenas os registros do FPSO P84
df_avanco_p85 = df_avanco[df_avanco['FPSO'] == 'P85']
# Ordena por data em ordem crescente para garantir que o último valor seja o mais recente
df_avanco_p85 = df_avanco_p85.sort_values(by='DATE')
# Obtém os últimos valores para cada PKG_DESCRIPTION
df_latest_p85 = df_avanco_p85.drop_duplicates(subset=['PKG_DESCRIPTION'], keep='last')
# Update df_db com os valores mais recentes
for index, row in df_latest_p85.iterrows():
    pkg_description = row['PKG_DESCRIPTION']
    plan = row['PLAN']
    real = row['REAL']
    desvio = row['DESVIO']
    # Update the corresponding rows in df_db
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'PLAN_P85'] = plan
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'REAL_P85'] = real
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'DESVIO_P85'] = desvio

# Substituir NaN por 0:
df_db['PLAN_P85'] = df_db['PLAN_P85'].fillna(0)
df_db['REAL_P85'] = df_db['REAL_P85'].fillna(0)
df_db['DESVIO_P85'] = df_db['DESVIO_P85'].fillna(0)

df_db['PLAN_P84'] = df_db['PLAN_P84'].fillna(0)
df_db['REAL_P84'] = df_db['REAL_P84'].fillna(0)
df_db['DESVIO_P84'] = df_db['DESVIO_P84'].fillna(0)


Atualizando informações dos Mapas de Suprimentos...
Processing DataFrame: df_mapa_topsideP84
Processing DataFrame: df_mapa_topsideP85
Atualizando informações do Orbis...
Atualizando informações do Communication Matrix...
Atualiza informações do Integra...
Atualiza informações das quantidades de equipamentos...
Atualiza informações do Avanço Geral...


## Avanço dos Pacotes

### Documentos em atraso com relação a última revisão da LD

In [8]:
from datetime import datetime

# Get today's date
today = datetime.now().date()

# Create a list to store delayed documents
delayed_docs = []

# Iterate through VDA documents
for index, row in df_LD.iterrows():
    client_document = row['CLIENT_DOCUMENT']
    
    # Verifica se PLANNED_DATE não é NaT
    if pd.notna(row['PLANNED_DATE']):
        planned_date = row['PLANNED_DATE'].date()
        
        # Check if document exists in df_integra_last
        doc_issued = client_document in df_integra_last['CODE'].values
        
        # Document is delayed if not issued and planned date has passed
        if not doc_issued and planned_date < today:
            delayed_docs.append({
                'CLIENT_DOCUMENT': client_document,
                'DOCUMENT_TITLE': row['DOCUMENT_TITLE'],
                'PLANNED_DATE': planned_date,
                'DAYS_DELAYED': (today - planned_date).days,
                'PKG_DESCRIPTION': row['PKG_DESCRIPTION']
            })

# Create DataFrame with delayed documents
df_atrasados = pd.DataFrame(delayed_docs)

# Se não houver documentos atrasados, crie um DataFrame vazio com as colunas necessárias
if df_atrasados.empty:
    df_atrasados = pd.DataFrame(columns=['CLIENT_DOCUMENT', 'DOCUMENT_TITLE', 'PLANNED_DATE', 'DAYS_DELAYED', 'PKG_DESCRIPTION'])

# Count documents by PKG_DESCRIPTION
if not df_atrasados.empty:
    df_atrasados_pkg = df_atrasados['PKG_DESCRIPTION'].value_counts().reset_index()
    df_atrasados_pkg.columns = ['PKG_DESCRIPTION', 'N_DOCS_EM_ATRASO']
    
    for index, row in df_atrasados_pkg.iterrows():
        pkg_description = row['PKG_DESCRIPTION']
        qty_docs = row['N_DOCS_EM_ATRASO']
        # Update the corresponding rows in df_db
        df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'N_DOCS_EM_ATRASO'] = qty_docs


In [9]:
print("Atualizando a quantidade de documentos emitidos...")
# Create a mapping dictionary from df_LD
pkg_map = df_LD.set_index('CLIENT_DOCUMENT')['PKG_DESCRIPTION'].to_dict()

# Add new PKG_DESCRIPTION column to df_integra_last
df_integra_last['PKG_DESCRIPTION'] = df_integra_last['CODE'].map(pkg_map)

# Count documents by PKG_DESCRIPTION
df_emitidos = df_integra_last['PKG_DESCRIPTION'].value_counts().reset_index()
df_emitidos.columns = ['PKG_DESCRIPTION', 'N_DOCS_EMITIDOS']

for index, row in df_emitidos.iterrows():
    pkg_description = row['PKG_DESCRIPTION']
    qty_docs = row['N_DOCS_EMITIDOS']
    
    # Update the corresponding rows in df_db
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'N_DOCS_EMITIDOS'] = qty_docs


Atualizando a quantidade de documentos emitidos...


In [10]:
print("Atualizando a quantidade de documentos...")

# Contagem mais robusta usando groupby e size
df_num_docs = df_LD.groupby('PKG_DESCRIPTION', dropna=False).size().reset_index(name='count')

# Update df_db with equipment quantities from df_num_docs
for index, row in df_num_docs.iterrows():
    pkg_description = row['PKG_DESCRIPTION']
    qty_docs = row['count']
    # Update the corresponding rows in df_db
    df_db.loc[df_db['PKG_DESCRIPTION'] == pkg_description, 'N_DOCS'] = qty_docs

# Verificar se todos os documentos foram contados
total_docs_original = len(df_LD)
total_docs_counted = df_num_docs['count'].sum()
print(f"Total de documentos original: {total_docs_original}")
print(f"Total de documentos contados: {total_docs_counted}")



Atualizando a quantidade de documentos...
Total de documentos original: 3180
Total de documentos contados: 3180


### Comentário em Atraso: Seatrium vs Petrobras

In [11]:
from datetime import datetime, timedelta
import numpy as np
from pandas.tseries.offsets import BusinessDay

# Get today's date
today = datetime.now().date()

# Set different tolerance days for each company
petrobras_tolerance_days = 8
seatrium_tolerance_days = 7

# Create lists to store delayed documents for each company
seatrium_delays = []
petrobras_delays = []

# Function to calculate business days between two dates
def business_days_between(start_date, end_date):
    # Convert to datetime if they are not
    if isinstance(start_date, str):
        start_date = datetime.strptime(start_date, '%Y-%m-%d').date()
    if isinstance(end_date, str):
        end_date = datetime.strptime(end_date, '%Y-%m-%d').date()
    
    # Calculate business days
    business_days = np.busday_count(
        start_date.strftime('%Y-%m-%d'),
        end_date.strftime('%Y-%m-%d')
    )
    return business_days

# Find the latest version for each document 'Code' in df_VDA
df_VDA_latest = df_VDA.loc[df_VDA.groupby('Code')['Version'].idxmax()]

# Check Petrobras delays (documents in VDA with status "EM ANALISE" for more than 8 business days)
for _, row in df_VDA_latest.iterrows():
    if row['Workflow Status'] == 'EM ANALISE':
        creation_date = row['Creation Date'].date()
        business_days = business_days_between(creation_date, today)
        if business_days > petrobras_tolerance_days:
            petrobras_delays.append({
                'document': row['Reference Document'],
                'vda': row['Code'],
                'title': row['Title'],
                'version': row['Version'],
                'workflow_status': row['Workflow Status'],
                'creation_date': row['Creation Date'],
                'delay': business_days
            })

# Check Seatrium delays (documents in LD but not in VDA and created more than 7 business days ago)
for _, row in df_LD.iterrows():
    client_document = row['CLIENT_DOCUMENT']
    # Check if document exists in df_integra_last
    doc_issued = client_document in df_integra_last['CODE'].values
    if doc_issued:
        # Check if document is in VDA
        doc_in_vda = client_document in df_VDA['Reference Document'].values
        if not doc_in_vda:
            # Get creation date from df_integra_last
            creation_date = df_integra_last[df_integra_last['CODE'] == client_document]['CREATION DATE'].iloc[0].date()
            business_days = business_days_between(creation_date, today)
            if business_days > seatrium_tolerance_days:
                seatrium_delays.append({
                    'document': client_document,
                    'title': row['TITLE'],
                    'planned_date': row['PLANNED_DATE'],
                    'discipline': row['Discipline'],
                    'comment': row['Comments'],
                    'creation_date': creation_date,
                    'delay': business_days
                })

# Create DataFrames for delayed documents
df_seatrium_delays = pd.DataFrame(seatrium_delays)
df_petrobras_delays = pd.DataFrame(petrobras_delays)

# Function to count delays for each package
def count_package_delays(pkg_row, df_delays):
    count = 0
    trigrama = pkg_row['Trigram']
    sistemas = pkg_row['System'].split(',') if pd.notna(pkg_row['System']) else []
    sistemas = [s.strip() for s in sistemas]
    for sistema in sistemas:
        if pd.notna(trigrama) and pd.notna(sistema):
            # Create regex pattern
            pattern = f".*{sistema}.*{trigrama}.*"
            # Count matching documents
            matching_docs = df_delays[df_delays['document'].str.contains(pattern, regex=True, na=False)]
            count += len(matching_docs)
    return count

# Mapeia coluna Discipline no df_petrobras_delays
# Create a mapping dictionary from df_LD
discipline_map = df_LD.set_index('CLIENT_DOCUMENT')['Discipline'].to_dict()
# Add new Discipline column to df_petrobras_delays
df_petrobras_delays['Discipline'] = df_petrobras_delays['document'].map(discipline_map)
# Sort by Discipline and delay
df_petrobras_delays = df_petrobras_delays.sort_values(['Discipline', 'delay'], ascending=[True, False])

# Add new columns to df_db
df_db['Atraso_KBR'] = df_db.apply(lambda row: count_package_delays(row, df_seatrium_delays), axis=1)
df_db['Atraso_PB'] = df_db.apply(lambda row: count_package_delays(row, df_petrobras_delays), axis=1)

# Definimos a nova ordem das colunas em uma lista
new_order = ['PKG_DESCRIPTION',	'QTY_EQUIPMENT', 'Discipline', 'Critical', 'System', 'Trigram', 'Vendor', 
             'MR_Number', 'MR_Emitida', 'TBE_Number', 'TBE_Emitida', 'PO_Number', 'KOM_DATE', 'PIM_DATE',	
             'DR_MOTORS_DATE', 'PLAN_DELIVERY_DATE', 'PO_Issue_Date_Plan', 'Required_On_Site_Date_Plan',	
             'Package_KBR',	'Package_Seatrium',	'Package_Expeditor', 'N_DOCS', 'N_DOCS_EMITIDOS', 
             'N_DOCS_EM_ATRASO', 'Atraso_KBR', 'Atraso_PB', 'PLAN_P84', 'REAL_P84', 'DESVIO_P84', 'PLAN_P85', 'REAL_P85', 'DESVIO_P85']


# Replace NaN with 0 for specific columns
columns_to_fill = ['N_DOCS', 'N_DOCS_EMITIDOS', 'N_DOCS_EM_ATRASO']
df_db[columns_to_fill] = df_db[columns_to_fill].fillna(0)

# Usamos essa lista para reordenar o DataFrame
df_db = df_db[new_order]

# Display updated df_db
# display(df_db)


#### Salva os dados em Arquivo Excel

In [12]:
# control_cesar = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2025-05-28_P84-P85_controle_cesar.xlsx'
from datetime import datetime

data_atual = datetime.now().strftime("%Y-%m-%d")
control_cesar = f"{data_atual} P84-P85_controle_cesar.xlsx"

# Salvando arquivos P84 e P85:
with pd.ExcelWriter(control_cesar) as writer:
    df_integra.to_excel(writer, sheet_name='integra_all_eng_docs', index=False)
    df_db.to_excel(writer, sheet_name='Resumo', index=False)
    df_all_equipment.to_excel(writer, sheet_name='equip', index=False)
    df_atrasados.to_excel(writer, sheet_name='docs_atrasados', index=False)
    df_petrobras_delays.to_excel(writer, sheet_name='petrobras_delays', index=False)
    df_seatrium_delays.to_excel(writer, sheet_name='seatrium_delays', index=False)
print(f'Arquivo {control_cesar} salvo em: ', Path.cwd())

Arquivo 2025-06-27 P84-P85_controle_cesar.xlsx salvo em:  C:\Users\ELXY\Documents\Codigos\Python\P84_85


## GRÁFICOS

In [13]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd

# Inicializar lista de figuras para armazenar os gráficos
figures = []

### Plota Gráfico com Número de Documentos e Documentos em Atraso

In [14]:
# Configuração do estilo do Seaborn
sns.set(style="whitegrid")

# Definir o tamanho da figura
plt.figure(figsize=(16, 6))

# Preparar os dados para plotagem
# Transformar os dados em formato long (melhor para Seaborn)
df_long = pd.melt(df_db,
                  id_vars=['PKG_DESCRIPTION'],
                  value_vars=['N_DOCS', 'N_DOCS_EMITIDOS', 'N_DOCS_EM_ATRASO'],
                  var_name='Tipo',
                  value_name='Número de Documentos')

# Criar o gráfico de barras lado a lado
sns.barplot(x='PKG_DESCRIPTION', 
            y='Número de Documentos', 
            hue='Tipo', 
            data=df_long,
            palette={'N_DOCS': 'blue', 'N_DOCS_EMITIDOS': 'green', 'N_DOCS_EM_ATRASO': 'red'})

# Adicionar rótulos e título
plt.xlabel('Package Description')
plt.ylabel('Number of Documents')
plt.title('Number of Documents by Package', fontdict={"size": 16})

# Rotacionar os rótulos do eixo x
plt.xticks(rotation=45, ha='right')

# Adicionar valores nas barras
for i in range(len(df_db['PKG_DESCRIPTION'])):
    for j, col in enumerate(['N_DOCS', 'N_DOCS_EMITIDOS', 'N_DOCS_EM_ATRASO']):
        valor = df_db[col][i]
        if valor != 0:
            plt.text(i + (j * 0.4) - 0.4, 
                     valor, 
                     str(valor), 
                     ha='center', 
                     va='bottom')

# Ajustar o layout
plt.tight_layout()

# Capturar a figura (se necessário)
figures.append(plt.gcf())
plt.close()


### Plota Gráfico dos Comentários em Atraso

In [15]:
# Configuração do estilo do Seaborn
sns.set(style="whitegrid")

# Definir o tamanho da figura
plt.figure(figsize=(16, 6))

# Usar uma paleta de cores predefinida do Seaborn
custom_palette = {
    'Atraso_KBR': mcolors.to_rgba('#1A5F7A', alpha=0.7),  # Azul com transparência
    'Atraso_PB': mcolors.to_rgba("#FA700E", alpha=0.7)    # Marrom com transparência
}

# Preparar os dados para plotagem
# Transformar os dados em formato long (melhor para Seaborn)
df_long = pd.melt(df_db, 
                  id_vars=['PKG_DESCRIPTION'], 
                  value_vars=['Atraso_KBR', 'Atraso_PB'],
                  var_name='Tipo',
                  value_name='Documentos em Atraso')

# Criar o gráfico de barras lado a lado usando a paleta de cores
sns.barplot(x='PKG_DESCRIPTION', 
            y='Documentos em Atraso', 
            hue='Tipo', 
            data=df_long,
            # palette={'Atraso_KBR': 'blue', 'Atraso_PB': 'orange'})
            palette=custom_palette)

# Adicionar rótulos e título
plt.xlabel('Package Description')
plt.ylabel('Overdue Documents')
plt.title('Overdue Comments by Package', fontdict={"size": 16})

# Rotacionar os rótulos do eixo x
plt.xticks(rotation=45, ha='right')

# Adicionar valores nas barras
for i in range(len(df_db['PKG_DESCRIPTION'])):
    for j, col in enumerate(['Atraso_KBR', 'Atraso_PB']):
        valor = df_db[col][i]
        if valor != 0:
            plt.text(i + (j * 0.4) - 0.2, 
                     valor, 
                     str(valor), 
                     ha='center', 
                     va='bottom')

# Ajustar o layout
plt.tight_layout()

# Capturar a figura
figures.append(plt.gcf())
plt.close()

### Plota Gráfico do Avanço Geral por Pacote - P84

In [16]:
# Configuração do estilo do Seaborn
sns.set(style="whitegrid")

# Definir o tamanho da figura
plt.figure(figsize=(16, 6))

# Preparar os dados para plotagem
# Transformar os dados em formato long (melhor para Seaborn)
df_long = pd.melt(df_db, 
                  id_vars=['PKG_DESCRIPTION'], 
                  value_vars=['PLAN_P84', 'REAL_P84'], 
                  var_name='Tipo', 
                  value_name='Progresso')

# Criar o gráfico de barras lado a lado
sns.barplot(x='PKG_DESCRIPTION', 
            y='Progresso', 
            hue='Tipo', 
            data=df_long,
            palette={'PLAN_P84': 'blue', 'REAL_P84': 'green'})

# Adicionar rótulos e título
plt.xlabel('Package Description')
plt.ylabel('Progress (%)')
plt.title('Overall Progress by Package - P84', fontdict={"size": 16})

# Rotacionar os rótulos do eixo x
plt.xticks(rotation=45, ha='right')

# Adicionar valores nas barras
for i in range(len(df_db['PKG_DESCRIPTION'])):
    for j, col in enumerate(['PLAN_P84', 'REAL_P84']):
        valor = df_db[col][i]
        if valor != 0:
            plt.text(i + (j * 0.4) - 0.2, 
                     valor, 
                     str(valor), 
                     ha='center', 
                     va='bottom')

# Ajustar o layout
plt.tight_layout()

# Capturar a figura (se necessário)
figures.append(plt.gcf())
plt.close()


### Plota Gráfico do Avanço Geral por Pacote - P85

In [17]:
# Configuração do estilo do Seaborn
sns.set(style="whitegrid")

# Definir o tamanho da figura
plt.figure(figsize=(16, 6))

# Preparar os dados para plotagem
# Transformar os dados em formato long (melhor para Seaborn)
df_long = pd.melt(df_db, 
                  id_vars=['PKG_DESCRIPTION'], 
                  value_vars=['PLAN_P85', 'REAL_P85'], 
                  var_name='Tipo', 
                  value_name='Progresso')

# Criar o gráfico de barras lado a lado
sns.barplot(x='PKG_DESCRIPTION', 
            y='Progresso', 
            hue='Tipo', 
            data=df_long,
            palette={'PLAN_P85': 'blue', 'REAL_P85': 'green'})

# Adicionar rótulos e título
plt.xlabel('Package Description')
plt.ylabel('Progress (%)')
plt.title('Overall Progress by Package - P85', fontdict={"size": 16})

# Rotacionar os rótulos do eixo x
plt.xticks(rotation=45, ha='right')

# Adicionar valores nas barras
for i in range(len(df_db['PKG_DESCRIPTION'])):
    for j, col in enumerate(['PLAN_P85', 'REAL_P85']):
        valor = df_db[col][i]
        if valor != 0:
            plt.text(i + (j * 0.4) - 0.2, 
                     valor, 
                     str(valor), 
                     ha='center', 
                     va='bottom')

# Ajustar o layout
plt.tight_layout()

# Capturar a figura (se necessário)
figures.append(plt.gcf())
plt.close()


### Gera relatório de avanço

In [18]:
import pandas as pd
import datetime
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle
from reportlab.lib.units import inch, cm

def generate_report(df_notes, figures, output_filename):
    """
    Gera um relatório PDF baseado em um dataframe de notas e figuras.
    
    Args:
    df_notes (pandas.DataFrame): DataFrame contendo as notas para o relatório 
    com colunas: 'PKG_DESCRIPTION', 'DATE', 'Tipo', 'Descricao'
    figures (list): Lista de objetos de figura (matplotlib ou caminho para imagens)
    output_filename (str): Nome do arquivo PDF de saída
    """
    
    # Configurar o documento em tamanho A4 com margens estreitas
    page_width, page_height = A4
    margin = 1.0 * cm # 1 cm de margem em todos os lados
    
    doc = SimpleDocTemplate(
        output_filename,
        pagesize=A4,
        rightMargin=margin,
        leftMargin=margin,
        topMargin=margin,
        bottomMargin=margin
    )
    
    styles = getSampleStyleSheet()
    
    # Calcular largura disponível para conteúdo
    content_width = page_width - (2 * margin)
    
    # Criar estilos personalizados com nomes únicos
    title_style = ParagraphStyle(
        name='CustomTitle',
        parent=styles['Heading1'],
        fontSize=16,
        alignment=1, # centralizado
        spaceAfter=12
    )
    
    subtitle_style = ParagraphStyle(
        name='CustomSubtitle',
        parent=styles['Heading2'],
        fontSize=12,
        alignment=1, # centralizado
        spaceAfter=12
    )
    
    section_title_style = ParagraphStyle(
        name='CustomSectionTitle',
        parent=styles['Heading2'],
        fontSize=14,
        spaceAfter=6
    )
    
    package_title_style = ParagraphStyle(
        name='CustomPackageTitle',
        parent=styles['Heading3'],
        fontSize=11,
        fontName='Helvetica-Bold',
        spaceAfter=4,
        spaceBefore=10
    )
    
    bullet_point_style = ParagraphStyle(
        name='CustomBulletPoint',
        parent=styles['Normal'],
        fontSize=10,
        leftIndent=20,
        bulletIndent=10,
        spaceAfter=2
    )
    
    sub_header_style = ParagraphStyle(
        name='CustomSubHeader',
        parent=styles['Normal'],
        fontSize=11,
        fontName='Helvetica-Bold',
        spaceAfter=2,
        spaceBefore=6
    )
    
    # Conteúdo do documento
    elements = []
    
    # Título
    elements.append(Paragraph("Progress Report - CÉSAR SILVESTRE", title_style))
    
    # Data
    current_date = datetime.datetime.now().strftime("%d/%m/%Y")
    elements.append(Paragraph(f"Date: {current_date}", subtitle_style))
    elements.append(Spacer(1, 0.25*inch))
    
    # Seção 1: Indicadores
    elements.append(Paragraph("1. Metrics:", section_title_style))
    elements.append(Spacer(1, 0.1*inch))
    
    # Adicionar as figuras uma embaixo da outra
    if len(figures) > 0:
        # Salvar temporariamente as figuras matplotlib se necessário
        image_paths = []
        for i, fig in enumerate(figures):
            if hasattr(fig, 'savefig'): # Se for uma figura matplotlib
                temp_path = f'temp_figure_{i}.png'
                fig.savefig(temp_path, dpi=100, bbox_inches='tight')
                image_paths.append(temp_path)
            else: # Se for um caminho para uma imagem
                image_paths.append(fig)
        
        # Definir dimensões para aproveitar melhor o espaço A4
        fig_width = content_width # Usa toda a largura disponível
        fig_height = content_width * 0.55 # Proporção aproximada para gráficos
        
        # Adicionar cada figura individualmente
        for img_path in image_paths:
            img = Image(img_path, width=fig_width, height=fig_height)
            elements.append(img)
            elements.append(Spacer(1, 0.2*inch)) # Espaço entre figuras
        
        elements.append(Spacer(1, 0.3*inch))
    
    # Seção 2: Comentários
    elements.append(Paragraph("2. Comments:", section_title_style))
    elements.append(Spacer(1, 0.1*inch))
    
    # Agrupar por pacote
    grouped_notes = df_notes.groupby('PKG_DESCRIPTION')
    
    # Processar cada pacote
    for pkg_description, pkg_group in grouped_notes:
        elements.append(Paragraph(pkg_description, package_title_style))
        
        # Processar Fatos Relevantes
        relevant_facts = pkg_group[pkg_group['Tipo'] == 'Fato Relevante']
        if not relevant_facts.empty:
            elements.append(Paragraph("<b><i>Relevant Facts:</i></b>", sub_header_style))
            for _, row in relevant_facts.iterrows():
                elements.append(Paragraph(f"{row['Descricao']}", bullet_point_style))
        
        # Processar Pontos de Atenção
        attention_points = pkg_group[pkg_group['Tipo'] == 'Ponto de Atenção']
        if not attention_points.empty:
            elements.append(Paragraph("<b><i>Concerns:</i></b>", sub_header_style))
            for _, row in attention_points.iterrows():
                elements.append(Paragraph(f"{row['Descricao']}", bullet_point_style))
        
        elements.append(Spacer(1, 0.1*inch))
    
    # Gerar o PDF
    doc.build(elements)
    print(f"Relatório gerado com sucesso: {output_filename}")


### Gera Relatórios

In [19]:
print("Gerando relatório gerencial...")

data_atual = datetime.datetime.now().strftime("%Y-%m-%d")
nome_relatorio = f"{data_atual} Relatório de Avanço - César.pdf"
print(f"Nome do relatório: {nome_relatorio}")
# Gere o relatório
generate_report(df_notes, figures, nome_relatorio)

Gerando relatório gerencial...
Nome do relatório: 2025-06-27 Relatório de Avanço - César.pdf
Relatório gerado com sucesso: 2025-06-27 Relatório de Avanço - César.pdf
